In [2]:
import pymongo
from pymongo import MongoClient
client = MongoClient()
client = MongoClient('localhost', 27017)
# client = MongoClient('mongodb://localhost:27017/')
db = client.db_cdiscount

In [3]:
import io
import bson                       # this is installed with the pymongo package
import matplotlib.pyplot as plt
from skimage.data import imread   # or, whatever image library you prefer
import multiprocessing as mp      # will come in handy due to the size of the data

In [4]:
import numpy as np

In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
import keras
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [6]:
first = 0
last = 7069896
batch = 4000
num_batch = int(last/batch)
# last = 82


In [7]:
epochs = 2
batch_size = 16
num_classes = db.cat_encode.count()

In [8]:
def simple_model(epochs=epochs, num_classes=num_classes, lrate=0.01, momentum=0.9):
    
    # Convolutional Layer
    model = Sequential()
    # Convolutional Layer
    model.add(Conv2D(40, (3,3), input_shape = (180,180,3), activation='relu'))

    # Pooling Layer
    model.add(MaxPooling2D(pool_size=(2, 1)))
    
    # Fully conected Layer
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))


    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

#     model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [9]:
def deeper_model(epochs=epochs, num_classes=num_classes, lrate=0.01, momentum=0.9):
    # Convolutional Layer
    model = Sequential()
    # Convolutional Layer
    model.add(Conv2D(10, (3,3), input_shape = (180,180,3), activation='relu'))
    model.add(Conv2D(13, 3, activation='relu', padding='same'))
    model.add(Conv2D(14, 3, activation='relu', padding='same'))

    # Pooling Layer
    model.add(MaxPooling2D(pool_size=(2, 1)))
    
    
    model.add(Conv2D(20, 3, activation='relu', padding='same'))    
    model.add(Conv2D(26, 3, activation='relu', padding='same'))
    
    # Fully conected Layer
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))


    lrate = 0.01
    decay = lrate/epochs
    sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)

#     model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
    

In [10]:
# model =simple_model(epochs=epochs, num_classes=num_classes)
model =deeper_model(epochs=epochs, num_classes=num_classes)


In [ ]:
import time

In [ ]:
start_time = time.time()

counter = 0
row_count = 0
for j in range(0,num_batch-1):
# for j in range(0,1):
    batch_time = time.time()
    a1 = int(list(np.linspace(first,last,num_batch))[j])
    an = int(list(np.linspace(first,last,num_batch))[j+1])
    
    lst_batch = []
    
#     print(a1,an)
    
    cur = db.train.find({})[a1:an]
    
    m= an-a1
        
    i = 0
#     pic_array
    while (cur.alive):
        idx = cur.next()
        dic = {}
        
        category_id =idx['category_id'] 
    #     print(category_id)

        cat = db.cat_encode.find_one({ "cat" : (category_id)}, {"cat" : 1.0, "_id" : 0})['cat']
    #     print(cat)
        
        picture = np.float32(imread(io.BytesIO(idx['imgs'][0]['picture']))/255.0)
        picture = picture.reshape(180,180,3)
        
        dic['picture'] = picture
        
        encode = db.cat_encode.find_one({ "cat" : (category_id)}, {"encode" : 1.0, "_id" : 0})['encode']
    #     print(encode)
        dic['encode'] = encode
        
        lst_batch.append(dic)
        
        i+=1
    X_batch = np.array([lst['picture'] for lst in lst_batch])
    Y_batch = np.array([lst['encode'] for lst in lst_batch])
    print('X batch size: ', X_batch.shape)
    print('Y batch size: ', Y_batch.shape)
    
    del(lst_batch)
        
    model.fit(X_batch, Y_batch, batch_size=batch_size, epochs=epochs, validation_split=0.2)
    
#     print(X_batch.shape[0])
    row_count = row_count + X_batch.shape[0]
    counter = counter + X_batch.shape[0]
    
    if(row_count >= 100000):
        print('row count: ',row_count)
        counter = counter + row_count
        model_name = 'model_' + str(counter) +'.h5'
        print(model_name)
        model.model.save('E://kaggle//Cdiscount//model//'+model_name)
        print('Model saved')
        row_count = 0
    
    print("--- %s seconds ---" % (time.time() - batch_time))
    print('rows completed: ', counter)
    
print("--- %s seconds ---" % (time.time() - start_time))
model.model.save('E://kaggle//Cdiscount//model//final_model.h5')

X batch size:  (4003, 180, 180, 3)
Y batch size:  (4003, 5270)
Train on 3202 samples, validate on 801 samples
Epoch 1/2
3202/3202 [==============================] - 52s - loss: 4.3998 - acc: 0.4263 - val_loss: 4.2029 - val_acc: 0.4132
Epoch 2/2
3202/3202 [==============================] - 44s - loss: 3.5531 - acc: 0.4341 - val_loss: 4.1436 - val_acc: 0.4132
--- 151.40980052947998 seconds ---
rows completed:  4003
X batch size:  (4003, 180, 180, 3)
Y batch size:  (4003, 5270)
Train on 3202 samples, validate on 801 samples
Epoch 1/2
3202/3202 [==============================] - 62s - loss: 4.0666 - acc: 0.4107 - val_loss: 3.8699 - val_acc: 0.4220
Epoch 2/2
3202/3202 [==============================] - 44s - loss: 3.5544 - acc: 0.4101 - val_loss: 3.8623 - val_acc: 0.4232
--- 152.4272756576538 seconds ---
rows completed:  8006
X batch size:  (4004, 180, 180, 3)
Y batch size:  (4004, 5270)
Train on 3203 samples, validate on 801 samples
Epoch 1/2
3203/3203 [==============================] - 46

3202/3202 [==============================] - 44s - loss: 5.0295 - acc: 0.2689 - val_loss: 5.5076 - val_acc: 0.2385
--- 125.5293436050415 seconds ---
rows completed:  80066
X batch size:  (4004, 180, 180, 3)
Y batch size:  (4004, 5270)
Train on 3203 samples, validate on 801 samples
Epoch 1/2
3203/3203 [==============================] - 44s - loss: 5.3256 - acc: 0.2460 - val_loss: 5.7059 - val_acc: 0.1848
Epoch 2/2
3203/3203 [==============================] - 45s - loss: 5.0899 - acc: 0.2466 - val_loss: 5.7335 - val_acc: 0.1935
--- 125.31051182746887 seconds ---
rows completed:  84070
X batch size:  (4003, 180, 180, 3)
Y batch size:  (4003, 5270)
Train on 3202 samples, validate on 801 samples
Epoch 1/2
3202/3202 [==============================] - 44s - loss: 5.6678 - acc: 0.1999 - val_loss: 5.9420 - val_acc: 0.1760
Epoch 2/2
3202/3202 [==============================] - 44s - loss: 5.4714 - acc: 0.2142 - val_loss: 5.9252 - val_acc: 0.1885
--- 122.95642304420471 seconds ---
rows completed:

3202/3202 [==============================] - 44s - loss: 5.2408 - acc: 0.2345 - val_loss: 5.4944 - val_acc: 0.2135
Epoch 2/2
3202/3202 [==============================] - 45s - loss: 5.1275 - acc: 0.2445 - val_loss: 5.3537 - val_acc: 0.2322
--- 125.87082028388977 seconds ---
rows completed:  260216
X batch size:  (4003, 180, 180, 3)
Y batch size:  (4003, 5270)
Train on 3202 samples, validate on 801 samples
Epoch 1/2
3202/3202 [==============================] - 44s - loss: 5.2441 - acc: 0.2414 - val_loss: 5.2690 - val_acc: 0.2509
Epoch 2/2
3202/3202 [==============================] - 44s - loss: 5.1349 - acc: 0.2470 - val_loss: 5.2567 - val_acc: 0.2584
--- 122.37172245979309 seconds ---
rows completed:  264219
X batch size:  (4004, 180, 180, 3)
Y batch size:  (4004, 5270)
Train on 3203 samples, validate on 801 samples
Epoch 1/2
3203/3203 [==============================] - 45s - loss: 5.2973 - acc: 0.2485 - val_loss: 5.0295 - val_acc: 0.2659
Epoch 2/2
3203/3203 [==========================

3202/3202 [==============================] - 45s - loss: 4.8574 - acc: 0.2726 - val_loss: 5.2560 - val_acc: 0.2509
--- 124.43317341804504 seconds ---
rows completed:  436362
X batch size:  (4004, 180, 180, 3)
Y batch size:  (4004, 5270)
Train on 3203 samples, validate on 801 samples
Epoch 1/2
3203/3203 [==============================] - 45s - loss: 5.0848 - acc: 0.2576 - val_loss: 5.3598 - val_acc: 0.2035
Epoch 2/2
3203/3203 [==============================] - 45s - loss: 4.9587 - acc: 0.2607 - val_loss: 5.3503 - val_acc: 0.2160
--- 125.00492095947266 seconds ---
rows completed:  440366
X batch size:  (4003, 180, 180, 3)
Y batch size:  (4003, 5270)
Train on 3202 samples, validate on 801 samples
Epoch 1/2
3202/3202 [==============================] - 45s - loss: 5.4823 - acc: 0.2036 - val_loss: 5.1811 - val_acc: 0.2372
Epoch 2/2
3202/3202 [==============================] - 44s - loss: 5.3428 - acc: 0.2111 - val_loss: 5.1817 - val_acc: 0.2310
--- 123.79050517082214 seconds ---
rows complet

3202/3202 [==============================] - 44s - loss: 5.2195 - acc: 0.2408 - val_loss: 5.2922 - val_acc: 0.2197
Epoch 2/2
3202/3202 [==============================] - 44s - loss: 5.1424 - acc: 0.2511 - val_loss: 5.3530 - val_acc: 0.2297
--- 134.27265191078186 seconds ---
rows completed:  616513
X batch size:  (4004, 180, 180, 3)
Y batch size:  (4004, 5270)
Train on 3203 samples, validate on 801 samples
Epoch 1/2
3203/3203 [==============================] - 44s - loss: 5.8849 - acc: 0.1605 - val_loss: 5.2886 - val_acc: 0.2397
Epoch 2/2
3203/3203 [==============================] - 44s - loss: 5.3933 - acc: 0.2170 - val_loss: 5.1606 - val_acc: 0.2484
--- 132.2530176639557 seconds ---
rows completed:  620517
X batch size:  (4003, 180, 180, 3)
Y batch size:  (4003, 5270)
Train on 3202 samples, validate on 801 samples
Epoch 1/2
3202/3202 [==============================] - 44s - loss: 5.3753 - acc: 0.2214 - val_loss: 5.3460 - val_acc: 0.2322
Epoch 2/2
3202/3202 [===========================

...

3202/3202 [==============================] - 73s - loss: 5.5740 - acc: 0.2227 - val_loss: 5.6832 - val_acc: 0.2372
Epoch 2/2
3202/3202 [==============================] - 44s - loss: 5.4846 - acc: 0.2274 - val_loss: 5.6797 - val_acc: 0.2247
--- 596.2918858528137 seconds ---
rows completed:  1489241
X batch size:  (4004, 180, 180, 3)
Y batch size:  (4004, 5270)
Train on 3203 samples, validate on 801 samples
Epoch 1/2
3203/3203 [==============================] - 55s - loss: 5.5927 - acc: 0.2195 - val_loss: 5.5866 - val_acc: 0.2235
Epoch 2/2
3203/3203 [==============================] - 45s - loss: 5.5186 - acc: 0.2267 - val_loss: 5.5102 - val_acc: 0.2260
--- 497.8437159061432 seconds ---
rows completed:  1493245
X batch size:  (4003, 180, 180, 3)
Y batch size:  (4003, 5270)
Train on 3202 samples, validate on 801 samples
Epoch 1/2
3202/3202 [==============================] - 44s - loss: 5.5625 - acc: 0.2214 - val_loss: 5.4110 - val_acc: 0.2335
Epoch 2/2
3202/3202 [==========================

c:\users\hamza\anaconda3\lib\site-packages\keras\callbacks.py:118: UserWarning: Method on_batch_end() is slow compared to the batch update (0.421214). Check your callbacks.
  % delta_t_median)


3202/3202 [==============================] - 107s - loss: 5.6062 - acc: 0.2146 - val_loss: 5.4079 - val_acc: 0.2285
Epoch 2/2
3202/3202 [==============================] - 44s - loss: 5.5129 - acc: 0.2152 - val_loss: 5.2972 - val_acc: 0.2472
--- 780.9133913516998 seconds ---
rows completed:  1625354
X batch size:  (4004, 180, 180, 3)
Y batch size:  (4004, 5270)
Train on 3203 samples, validate on 801 samples
Epoch 1/2
3203/3203 [==============================] - 54s - loss: 5.5626 - acc: 0.2157 - val_loss: 5.4181 - val_acc: 0.2210
Epoch 2/2
3203/3203 [==============================] - 44s - loss: 5.4782 - acc: 0.2239 - val_loss: 5.3296 - val_acc: 0.2247
--- 514.5042533874512 seconds ---
rows completed:  1629358
X batch size:  (4003, 180, 180, 3)
Y batch size:  (4003, 5270)
Train on 3202 samples, validate on 801 samples
Epoch 1/2
3202/3202 [==============================] - 76s - loss: 5.5965 - acc: 0.2111 - val_loss: 5.6026 - val_acc: 0.2022
Epoch 2/2
3202/3202 [=========================

c:\users\hamza\anaconda3\lib\site-packages\keras\callbacks.py:118: UserWarning: Method on_batch_end() is slow compared to the batch update (0.237576). Check your callbacks.
  % delta_t_median)


3203/3203 [==============================] - 85s - loss: 5.5405 - acc: 0.2198 - val_loss: 5.8192 - val_acc: 0.2072
Epoch 2/2
3203/3203 [==============================] - 45s - loss: 5.4543 - acc: 0.2267 - val_loss: 5.8996 - val_acc: 0.2085
--- 916.9995400905609 seconds ---
rows completed:  1689408
X batch size:  (4003, 180, 180, 3)
Y batch size:  (4003, 5270)
Train on 3202 samples, validate on 801 samples
Epoch 1/2
3202/3202 [==============================] - 81s - loss: 5.6190 - acc: 0.2099 - val_loss: 5.3840 - val_acc: 0.2397
Epoch 2/2
3202/3202 [==============================] - 44s - loss: 5.5270 - acc: 0.2183 - val_loss: 5.4158 - val_acc: 0.2447
--- 934.6277885437012 seconds ---
rows completed:  1693411
X batch size:  (4003, 180, 180, 3)
Y batch size:  (4003, 5270)
Train on 3202 samples, validate on 801 samples
Epoch 1/2
3202/3202 [==============================] - 61s - loss: 5.7060 - acc: 0.2111 - val_loss: 5.5791 - val_acc: 0.2272
Epoch 2/2
3202/3202 [==========================

In [ ]:
# start_time = time.time()

# counter = 0
# for j in range(0,num_batch-1):
# # for j in range(0,1):
    
#     a1 = int(list(np.linspace(first,last,num_batch))[j])
#     an = int(list(np.linspace(first,last,num_batch))[j+1])
    
#     lst_batch = []
    
# #     print(a1,an)
    
#     cur = db.train.find({})[a1:an]
    
#     m= an-a1
        
#     i = 0
# #     pic_array
#     while (cur.alive):
#         idx = cur.next()
#         dic = {}
        
#         category_id =idx['category_id'] 
#     #     print(category_id)

#         cat = db.cat_encode.find_one({ "cat" : (category_id)}, {"cat" : 1.0, "_id" : 0})['cat']
#     #     print(cat)
        
#         picture = np.float32(imread(io.BytesIO(idx['imgs'][0]['picture']))/255.0)
#         picture = picture.reshape(180,180,3)
        
#         dic['picture'] = picture
        
#         encode = db.cat_encode.find_one({ "cat" : (category_id)}, {"encode" : 1.0, "_id" : 0})['encode']
#     #     print(encode)
#         dic['encode'] = encode
        
#         lst_batch.append(dic)
        
#         i+=1
#     X_batch = np.array([lst['picture'] for lst in lst_batch])
#     Y_batch = np.array([lst['encode'] for lst in lst_batch])
#     print(X_batch.shape)
#     print(Y_batch.shape)
    
#     del(lst_batch)
    
#     model.fit(X_batch, Y_batch, batch_size=batch_size, epochs=epochs, validation_split=0.2)
    
# print("--- %s seconds ---" % (time.time() - start_time))